# Reweighting MC simulation to data using a NN
Reweighting MC simulations to data is a common task used to improve the modelling. The most common practice is to reweight a single variable of particular importance in an analysis. This example demonstrates how a NN can be used to perform the reweighting task by considering multiple variables together, which improves the modelling across multiple variables and can account for correlations between variables.

**Many thanks to Michele Faucci Giannelli, Marilena Bandieramonte and Martina Javurkova**

In [1]:
!pip install -U tensorflow
!pip install -U tensorflow-probability
!pip install -U matplotlib

  Obtaining dependency information for tensorflow from https://files.pythonhosted.org/packages/e3/ba/aa8a76eff5c20761b0361a5b4c9fccb8742c29a82adba7a8ad8ae819984e/tensorflow-2.15.0.post1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
  Obtaining dependency information for absl-py>=1.0.0 from https://files.pythonhosted.org/packages/01/e4/dc0a1dcc4e74e08d7abedab278c795eef54a224363bb18f5692f416d834f/absl_py-2.0.0-py3-none-any.whl.metadata
  Obtaining dependency information for flatbuffers>=23.5.26 from https://files.pythonhosted.org/packages/6f/12/d5c79ee252793ffe845d58a913197bfa02ae9a0b5c9bc3dc4b58d477b9e7/flatbuffers-23.5.26-py2.py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 12.3 MB/s eta 0:00:00
  Obtaining dependency information for h5py>=2.9.0 from https://files.pythonhosted.org/packages/3b/d3/ecb4b3d2ec2c84132987e5f12ab1408f455bec1d90cd5bc408ebf37800f5/h5py-3.10.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.met

In [ ]:
import numpy as np
from scipy import stats
import tensorflow as tf
import tensorflow_probability as tfp
from matplotlib import pyplot as plt
tfd = tfp.distributions

## Model the variables with models for data and MC that are similar but have differences that we want to reweight. 
This section generates the data and MC samples.

In [ ]:
data_model_var1 = tfd.mixture.Mixture(tfd.Categorical(probs=[0.4, 0.6]),
            components=[
              tfd.Normal(loc=0., scale=0.1),
              tfd.Normal(loc=0., scale=0.8),
            ])
data_model_var2 = tfd.Exponential(0.2)

mc_model_var1 = tfd.mixture.Mixture(tfd.Categorical(probs=[0.3, 0.7]),
            components=[
              tfd.Normal(loc=0.1, scale=0.1),
              tfd.Normal(loc=0., scale=1),
            ])
mc_model_var2 = tfd.Exponential(0.4)

## Generate the random data from the model
The generated data has 3 columns. The first two are two observed variables, the third is the label: 0 for data, 1 for MC

In [ ]:
NSAMPLE = 100000
data_var1 = data_model_var1.sample(NSAMPLE).numpy()
data_var2 = data_model_var2.sample(NSAMPLE).numpy()
mc_var1 = mc_model_var1.sample(NSAMPLE).numpy()
mc_var2 = mc_model_var2.sample(NSAMPLE).numpy()

Add a rotation to create correlations and make these correlations different in data and MC.

In [ ]:
data_var1, data_var2 = 0.7 * data_var1 + 0.3 * data_var2, 0.3 * data_var1 + 0.7 * data_var2

mc_var1, mc_var2 = 0.9 * mc_var1 + 0.1 * mc_var2, 0.1 * mc_var1 + 0.9 * mc_var2

Merge the variables in a single vector, put a flag as the last element (0/1 for data/MC respectively).

In [ ]:
data = np.vstack([data_var1, data_var2, np.zeros(NSAMPLE)]).T
mc = np.vstack([mc_var1, mc_var2, np.ones(NSAMPLE)]).T

#### Apply selections to the data
Here one can implement selections to focus on specific parts of the distribution. It is especially challenging to model the tails of distributions, so these can be reduced so that the training focuses on the bulk of the distribution.

In [ ]:
#data_cut1 = data[data[:,0]>-5]
#data_cut2 = data_cut1[data_cut1[:,0]<10]
#data_cut3 = data_cut2[data_cut2[:,1]<20]
#mc_cut1 = mc[mc[:,0]>-5]
#mc_cut2 = mc_cut1[mc_cut1[:,0]<10]
#mc_cut3 = mc_cut2[mc_cut2[:,1]<20]
#data_sel = data_cut3
#mc_sel = mc_cut3

#### Create training and testing samples
Create a sample by mixing and shuffling data and MC. 

In [ ]:
#sample = np.vstack([data_sel, mc_sel])
sample = np.vstack([data, mc])
np.random.shuffle(sample)
NSAMPLE=len(sample)

Divide the dataset into training and testing samples by splitting them in half.

In [ ]:
training_sample = sample[int(NSAMPLE / 2):]
testing_sample = sample[:int(NSAMPLE / 2)]

X_train = training_sample[:, :-1]
Y_train = training_sample[:, -1]

## Plot the variables before the correction

In [ ]:
testing_sample_mc = testing_sample[testing_sample[:, -1] == 1]
testing_sample_data = testing_sample[testing_sample[:, -1] == 0]

fig, axs = plt.subplots(1, 2, figsize=(13, 6))
bins = np.linspace(-2, 4, 100)
axs[0].hist(testing_sample_mc[:, 0], bins=bins, histtype='step', label='MC', lw=2)
axs[0].hist(testing_sample_data[:, 0], bins=bins, histtype='step', label='data', lw=2)
axs[0].legend(loc=0, fontsize=14)
axs[0].set_title('var1', fontsize=14)

bins = np.linspace(-1, 7, 100)
axs[1].hist(testing_sample_mc[:, 1], bins=bins, histtype='step', lw=2)
axs[1].hist(testing_sample_data[:, 1], bins=bins, histtype='step', lw=2)
axs[1].set_title('var2', fontsize=14)


plt.show()

## Optimize the classifier, optimizing the cross entropy
Optimize a classifier to distinguish between data and MC

In [ ]:
classifier = tf.keras.Sequential([
    tf.keras.Input(X_train.shape[1]),
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

classifier.compile(loss='binary_crossentropy', metrics='accuracy')
classifier.fit(X_train, Y_train, epochs=10)

## Compute the weight on the test sample (the value is close to 1)

The weight is just the ratio between

$$w = \frac{P[data|q]}{P[MC|q]}$$

Where $q$ is our variable(s). Usually this is estimated with a multi-dimensional histogram, or just ignoring correlation using a set of 1D histograms.

A perfect classifier will return $y=P[data|x]$. Of course $P[MC|x] = 1 - P[data|x]$, where $x$ are the feature of one event. So:

$$w = \frac{y}{1 - y}$$

Note that if we have data very similar to MC we will get a very poor classifier, which will output 0.5, and so the weight will be 1.

An additional thing to do (not done here), is to calibrate the classifier.

In [ ]:
print(len(testing_sample_mc))
weights = classifier.predict(testing_sample_mc[:, :-1])
weights = weights / (1 - weights)    

## Distribution of the weights from the NN

In [ ]:
fig, ax = plt.subplots()
ax.hist(weights, bins=np.linspace(0, 1, 100))
plt.show()

## Apply the weights and plot the distributions

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(13, 6))
bins = np.linspace(-2, 4, 100)
axs[0].hist(testing_sample_mc[:, 0], weights=1/weights, bins=bins, histtype='step', label='MC reweighted', lw=2)
axs[0].hist(testing_sample_data[:, 0], bins=bins, histtype='step', label='data', lw=2)
axs[0].hist(testing_sample_mc[:, 0], bins=bins, histtype='step', label='MC', ls='--')
axs[0].legend(loc=0, fontsize=14)
axs[0].set_title('var1', fontsize=14)

bins = np.linspace(-1, 7, 100)
axs[1].hist(testing_sample_mc[:, 1], weights=1/weights, bins=bins, histtype='step', lw=2)
axs[1].hist(testing_sample_data[:, 1], bins=bins, histtype='step', lw=2)
axs[1].hist(testing_sample_mc[:, 1], bins=bins, histtype='step', label='MC', ls='--')
axs[1].set_title('var2', fontsize=14)

plt.show()

## Look at the correlation
Check to see if the reweighed MC reproduces the correlation in data!

In [ ]:
fig, axs = plt.subplots(1, 3, figsize=(16, 5))
bins = [np.linspace(-2, 2, 50), np.linspace(0., 10, 50)]
axs[0].hist2d(testing_sample_data[:, 0], testing_sample_data[:, 1], bins=bins)
axs[1].hist2d(testing_sample_mc[:, 0], testing_sample_mc[:, 1], bins=bins)
axs[2].hist2d(testing_sample_mc[:, 0], testing_sample_mc[:, 1], bins=bins, weights=1/np.squeeze(weights))
axs[0].set_title('data', fontsize=14)
axs[1].set_title('MC', fontsize=14)
axs[2].set_title('MC reweighted', fontsize=14)
for ax in axs:
    ax.set_xlabel('var1', fontsize=14)
    ax.set_ylabel('var2', fontsize=14)
plt.show()

## Compare effective statistics with size of the sample
We loose a lot of effective statistics. Try running more events to improve the modelling.

In [ ]:
w = 1/np.squeeze(weights)
np.sum(w) ** 2 / np.sum(w ** 2)

In [ ]:
len(w)

## Exercises
Try making changes to the configuration of the NN, the input variable data and to see if you can improve the modelling of the bulk of the distribution.
1. Implement selections to reduce the tails and reduce biases. 
2. Try increasing the number of epochs. 
3. Try changing the activation function on the NN, for example from relu to swish.
4. Try changing the structure of the NN. 
5. Increasing the sample size. Note if you make it too large it will take long to run. 